## 第一部分BitFit实战



In [11]:
import os
# 切换到国内镜像站，速度快且稳定
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from datasets import Dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer
print(1)

1


In [12]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")#要先把数据ds.save_to_disk才能用load_from_disk，现在就是这个情况
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [13]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [14]:
def process_func(example):
    Max_LENGTH = 256
    input_ids,attention_mask,labels = [],[],[]
    instruction = tokenizer("\n".join(['Human:'+ example["instruction"],example["input"]]).strip() + "\n\nAssistant:")
    response = tokenizer(example["output"]+tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"])+response["input_ids"]#-100表示不管，只学答案部分
    if len(input_ids)>Max_LENGTH:
        input_ids = input_ids[:Max_LENGTH]
        attention_mask = attention_mask[:Max_LENGTH]
        labels = labels[:Max_LENGTH]
    return{
        "input_ids":input_ids,
        "attention_mask":attention_mask,
        "labels":labels
    }


In [15]:
tokenized_ds = ds.map(process_func,remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [16]:
tokenizer.decode(tokenized_ds[1]["input_ids"])


'Human:解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant:4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

Python 3 中，filter() 返回的是一个 迭代器（Iterator），迭代器需要类似于遍历操作比如list去激活才会使用，而tokenizer.decode(token_ids) 函数，它的参数 token_ids 明确要求必须是 序列（Sequence） 类型，比如 List[int]（整数列表）、Tensor 或者 Numpy Array

In [17]:
tokenizer.decode(list(filter(lambda x:x!=-100,tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

model.parameters()：返回的是一个个 Tensor（张量/矩阵）。

比如：一个形状为 (3, 4) 的矩阵。param.numel()：意思是 Number of Elements（元素的数量）
对于 (3, 4) 的矩阵，numel() 返回 12。它是在数“箱子里有几个苹果”。
low_cpu_mem_usage=True（低 CPU 内存占用模式）加载模型节省内存


In [18]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh",low_cpu_mem_usage=True)
sum(param.numel() for param in model.parameters())

1303111680

In [19]:
num_param = 0
for name,param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param += param.numel()
num_param

544768

In [20]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_eval_batch_size=1,#在验证阶段每张显卡一次读取多少条数据
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,#每八次更新一次梯度
    logging_steps=10,#每训练多少打印一次日志，主要是loss
    num_train_epochs=1,#整个完整训练集学几遍
    fp16=False,
    gradient_checkpointing=False,
)

Trainer并没有克隆一个新的模型去训练，而是直接修改你传进去的那个 model 对象
每一次 Step（每一个 Batch），model 里的参数（Weights）都在被更新。

如果你在训练途中点了“停止”按钮（Interrupt Kernel），内存里的 model 就会停留在你点击那一瞬间的状态。

In [21]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True),
)

/var/folders/vt/2rwrwr1j1hn9724wbbsr7jm80000gn/T/ipykernel_6389/3248672971.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
import torch

# 1. 自动选择设备：如果有苹果加速用 mps，否则用 cpu
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [23]:
trainer.train()

/Users/liuzhen/Documents/PEFT Study/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.717900
20,2.711400
30,2.730300
40,2.675800
50,2.624700
60,2.621200
70,2.533700
80,2.589300
90,2.461800
100,2.578100


KeyboardInterrupt: 

In [24]:
import torch
import os

# 1. 定义保存路径
output_dir = "./chatbot_bitfit_final"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 2. 核心逻辑：只提取 requires_grad=True 的参数 (也就是 Bias)
# 这里的逻辑是：遍历模型所有参数，谁刚才参与了训练，就把谁拿出来
trainable_params = {
    name: param.cpu() # 记得搬回 CPU，防止下次加载时设备不兼容
    for name, param in model.named_parameters() 
    if param.requires_grad
}

# 3. 保存为 PyTorch 文件
save_path = os.path.join(output_dir, "bitfit_weights.pt")
torch.save(trainable_params, save_path)

# 4. 顺便保存一下 tokenizer，方便以后直接用这个目录
tokenizer.save_pretrained(output_dir)

print(f"✅ 模型保存成功！")
print(f"💾 权重文件位置: {save_path}")
print(f"📉 文件大小: {os.path.getsize(save_path) / 1024 / 1024:.2f} MB")

✅ 模型保存成功！
💾 权重文件位置: ./chatbot_bitfit_final/bitfit_weights.pt
📉 文件大小: 2.12 MB


In [30]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 1. 准备工作：自动定设备 (Mac用mps)
device = "mps" if torch.backends.mps.is_available() else "cpu"
base_path = "Langboat/bloom-1b4-zh"
save_path = "./chatbot_bitfit_final" # 你刚才保存的目录

# 2. 加载底座模型 + 分词器
model = AutoModelForCausalLM.from_pretrained(base_path, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(save_path)

# 3. 【核心】加载 BitFit 权重并覆盖
# strict=False 意思是：只更新文件里有的参数(Bias)，缺的参数(Weight)就用底座模型原来的，不报错，map_location表示用cpu加载模型
model.load_state_dict(torch.load(f"{save_path}/bitfit_weights.pt", map_location="cpu"), strict=False)
model = model.to(device)

# --- 方式一：手动推理 (Manual) ---
print(">>> 手动推理:")
ipt = tokenizer("Human: 考试有哪些技巧？\n\nAssistant: ", return_tensors="pt").to(device)
# [0]是解包batch，skip_special_tokens是去掉了</s>等符号
print(tokenizer.decode(model.generate(**ipt, max_length=512, do_sample=True)[0], skip_special_tokens=True))

# --- 方式二：Pipeline (封装版) ---
print("\n>>> Pipeline推理:")
# Mac上 device 直接传字符串 "mps" 即可
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
print(pipe("Human: 考试有哪些技巧？\n\nAssistant: ", max_new_tokens=256, do_sample=True)[0]['generated_text'])#max_length是提示词加回答长度，max_new_tokens是回答长度

>>> 手动推理:


Device set to use mps


Human: 考试有哪些技巧？

Assistant: 考试有许多技巧，比如对答题时间的充分利用：尽量保持整块答题时间来答题；利用草稿纸记录答案要点等。此外，要做到认真阅读题目中的所有信息，在听不懂的情况下不要急于下笔。最重要的是，保持良好状态，不要焦虑，努力迎接下次考试。

>>> Pipeline推理:
Human: 考试有哪些技巧？

Assistant: 考试技巧就是掌握学习方法，提高学习效率。学习效率高，才能提高学习效果；反之，学习效果不好，学习效率自然也难以提高。考试时，考生除了掌握考试技巧，还需要遵循循序渐进的学习原则。


In [31]:
# 查看模型自带的生成配置
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 3
}

